In [90]:
#import necessary libraries
from flask import Flask
from flask_restful import Resource, Api, reqparse
from google.cloud import bigquery
import pandas_gbq
import pandas as pd
import json
import os
from geojson import Feature, FeatureCollection, Point, Polygon
import geopandas
from shapely.geometry import Point, Polygon
import numpy as np

In [14]:
#google credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hurricane -32cb24436d46.json"

# #create Flass app instance
# app = Flask(__name__)
# api = Api(app)


#setup connection to BigAssQuery
client = bigquery.Client()


# @app.route("/")
# def index():

query = """
    SELECT year, latitude, longitude, sea_surface_temp
    FROM `bigquery-public-data.noaa_icoads.icoads_core_*` 
    WHERE year >=1851 
    AND latitude <=50
    AND latitude >=0
    AND longitude <=-50
    AND longitude >=-100
    ORDER BY year
    LIMIT 100000
"""

# query_res = client.query(query) 

In [64]:
df = pandas_gbq.read_gbq(query)

Downloading: 100%|██████████| 100000/100000 [00:03<00:00, 31701.48rows/s]


In [66]:
df.to_csv(r'sstSmall.csv',index=False)
dfDownload = df

In [67]:
lat_bins=range(0, 51, 10)
dfDownload['latitudeSquare'] = pd.cut(dfDownload['latitude'], bins=lat_bins)

In [68]:
long_bins=range(0, 51, 10)
dfDownload['longitudeSquare'] = pd.cut(dfDownload['longitude'], bins=lat_bins)

In [69]:
dfSquare=dfDownload.groupby(['year','latitudeSquare','longitudeSquare']).agg({'sea_surface_temp': ['min','max','mean','median']})
dfSquare.head(100)

sea_surface_temp                        
                                                 min   max       mean median
year latitudeSquare longitudeSquare                                         
1851 (0, 10]        (-100, -90]                 19.4  26.7  23.557895   23.3
                    (-90, -80]                  21.7  27.8  25.121739   24.4
                    (-80, -70]                  25.0  28.9  26.983333   26.4
                    (-70, -60]                   NaN   NaN        NaN    NaN
                    (-60, -50]                  26.1  28.3  26.800000   26.4
...                                              ...   ...        ...    ...
1854 (40, 50]       (-100, -90]                  NaN   NaN        NaN    NaN
                    (-90, -80]                   NaN   NaN        NaN    NaN
                    (-80, -70]                   2.8  24.4  14.880159   15.6
                    (-70, -60]                  -2.2  27.8  14.705409   15.6
                    (-60, -50]                  -2.2  26.7  14.707746   15.6

[100 rows x 4 columns]

In [70]:
dfProcessed = dfSquare.reset_index()
dfProcessed.head()

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                    min   max       mean   
0  1851        (0, 10]     (-100, -90]             19.4  26.7  23.557895   
1  1851        (0, 10]      (-90, -80]             21.7  27.8  25.121739   
2  1851        (0, 10]      (-80, -70]             25.0  28.9  26.983333   
3  1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4  1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   

          
  median  
0   23.3  
1   24.4  
2   26.4  
3    NaN  
4   26.4

In [71]:
dfProcessed.to_csv(r'sstProcessedSmall.csv',index=False)

In [72]:
dfFromCSV = pd.read_csv('sstProcessedSmall.csv')

In [73]:
dfDrop=dfFromCSV.dropna()
dfDrop

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842105,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.12173913043478,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
5,1851.0,"(0, 10]","(-60, -50]",26.1,28.3,26.8,26.4
6,1851.0,"(10, 20]","(-100, -90]",24.4,28.3,27.381818181818186,27.8
...,...,...,...,...,...,...,...
169,1857.0,"(30, 40]","(-70, -60]",0.6,29.4,21.113317073170712,21.1
170,1857.0,"(30, 40]","(-60, -50]",10.0,28.3,20.839299920445445,20.6
173,1857.0,"(40, 50]","(-80, -70]",1.1,26.1,13.636363636363638,14.4
174,1857.0,"(40, 50]","(-70, -60]",-2.8,26.7,13.36372332597497,13.9


In [74]:
dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r'(', '').astype(str)
dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r'(', '').astype(str)
dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r']', '').astype(str)
dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r']', '').astype(str)
dfStrReplace=dfDrop
dfStrReplace

<ipython-input-74-ccbef9c27e58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r'(', '').astype(str)
<ipython-input-74-ccbef9c27e58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r'(', '').astype(str)
<ipython-input-74-ccbef9c27e58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8
...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9


In [75]:
new = dfStrReplace["latitudeSquare"].str.split(",", n = 1, expand = True) 
dfStrReplace["latitudeLeft"]= new[0] 
dfStrReplace["latitudeRight"]= new[1] 
dfStrReplace

<ipython-input-75-a4e8e887e6a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["latitudeLeft"]= new[0]
<ipython-input-75-a4e8e887e6a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["latitudeRight"]= new[1]


,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3,latitudeLeft,latitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20
...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50


In [76]:
new = dfStrReplace["longitudeSquare"].str.split(",", n = 1, expand = True) 
dfStrReplace["longitudeLeft"]= new[0] 
dfStrReplace["longitudeRight"]= new[1] 
dfStrReplace

<ipython-input-76-7b66097bf524>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["longitudeLeft"]= new[0]
<ipython-input-76-7b66097bf524>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["longitudeRight"]= new[1]


,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10,-100,-90
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10,-90,-80
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10,-80,-70
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10,-60,-50
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20,-100,-90
...,...,...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40,-70,-60
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40,-60,-50
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50,-80,-70
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50,-70,-60


In [123]:
dfSplit=dfStrReplace
dfSplit

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10,-100,-90
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10,-90,-80
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10,-80,-70
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10,-60,-50
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20,-100,-90
...,...,...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40,-70,-60
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40,-60,-50
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50,-80,-70
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50,-70,-60


In [124]:
dfFinal= dfSplit.rename(columns={"sea_surface_temp": "sstMin", "sea_surface_temp.1": "sstMax", 
                       "sea_surface_temp.2": "sstMean", "sea_surface_temp.3": "sstMedian"})

In [125]:
dfFinal


,year,latitudeSquare,longitudeSquare,sstMin,sstMax,sstMean,sstMedian,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10,-100,-90
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10,-90,-80
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10,-80,-70
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10,-60,-50
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20,-100,-90
...,...,...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40,-70,-60
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40,-60,-50
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50,-80,-70
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50,-70,-60


In [127]:
dfFinal.dtypes
dfLatLong=dfFinal

In [132]:
dfLatLong['point1']= dfLatLong['latitudeLeft'] + ", " + dfLatLong['longitudeLeft']
dfLatLong['point2']= dfLatLong['latitudeLeft']+ ", " + dfLatLong['longitudeRight']
dfLatLong['point3']= dfLatLong['latitudeRight']+ ", " + dfLatLong['longitudeRight']
dfLatLong['point4']= dfLatLong['latitudeRight']+ ", " + dfLatLong['longitudeLeft']
dfLatLong['point5']= dfLatLong['latitudeLeft']+ ", " + dfLatLong['longitudeLeft']
dfLatLong['uniqueKey']=

,year,latitudeSquare,longitudeSquare,sstMin,sstMax,sstMean,sstMedian,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight,point1,point2,point3,point4,point5
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10,-100,-90,"0, -100","0, -90","10, -90","10, -100","0, -100"
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10,-90,-80,"0, -90","0, -80","10, -80","10, -90","0, -90"
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10,-80,-70,"0, -80","0, -70","10, -70","10, -80","0, -80"
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10,-60,-50,"0, -60","0, -50","10, -50","10, -60","0, -60"
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20,-100,-90,"10, -100","10, -90","20, -90","20, -100","10, -100"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40,-70,-60,"30, -70","30, -60","40, -60","40, -70","30, -70"
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40,-60,-50,"30, -60","30, -50","40, -50","40, -60","30, -60"
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50,-80,-70,"40, -80","40, -70","50, -70","50, -80","40, -80"
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50,-70,-60,"40, -70","40, -60","50, -60","50, -70","40, -70"


In [133]:
dfLatLong["sstMin"] = dfLatLong.sstMin.astype(float)
dfLatLong["sstMax"] = dfLatLong.sstMax.astype(float)
dfLatLong["sstMean"] = dfLatLong.sstMin.astype(float)
dfLatLong["sstMedian"] = dfLatLong.sstMax.astype(float)
dfLatLong["latitudeLeft"] = dfLatLong.latitudeLeft.astype(int)
dfLatLong["latitudeRight"] = dfLatLong.latitudeRight.astype(int)
dfLatLong["longitudeLeft"] = dfLatLong.longitudeLeft.astype(int)
dfLatLong["longitudeRight"] = dfLatLong.longitudeRight.astype(int)
dfLatLong["latitudeSquare"] = dfLatLong.longitudeLeft.astype(str)
dfLatLong["longitudeSquare"] = dfLatLong.longitudeRight.astype(str)
dfLatLong

,year,latitudeSquare,longitudeSquare,sstMin,sstMax,sstMean,sstMedian,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight,point1,point2,point3,point4,point5
1,1851.0,-100,-90,19.4,26.7,19.4,26.7,0,10,-100,-90,"0, -100","0, -90","10, -90","10, -100","0, -100"
2,1851.0,-90,-80,21.7,27.8,21.7,27.8,0,10,-90,-80,"0, -90","0, -80","10, -80","10, -90","0, -90"
3,1851.0,-80,-70,25.0,28.9,25.0,28.9,0,10,-80,-70,"0, -80","0, -70","10, -70","10, -80","0, -80"
5,1851.0,-60,-50,26.1,28.3,26.1,28.3,0,10,-60,-50,"0, -60","0, -50","10, -50","10, -60","0, -60"
6,1851.0,-100,-90,24.4,28.3,24.4,28.3,10,20,-100,-90,"10, -100","10, -90","20, -90","20, -100","10, -100"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,1857.0,-70,-60,0.6,29.4,0.6,29.4,30,40,-70,-60,"30, -70","30, -60","40, -60","40, -70","30, -70"
170,1857.0,-60,-50,10.0,28.3,10.0,28.3,30,40,-60,-50,"30, -60","30, -50","40, -50","40, -60","30, -60"
173,1857.0,-80,-70,1.1,26.1,1.1,26.1,40,50,-80,-70,"40, -80","40, -70","50, -70","50, -80","40, -80"
174,1857.0,-70,-60,-2.8,26.7,-2.8,26.7,40,50,-70,-60,"40, -70","40, -60","50, -60","50, -70","40, -70"


In [134]:
dfLatLong.dtypes

year               float64
latitudeSquare      object
longitudeSquare     object
sstMin             float64
sstMax             float64
sstMean            float64
sstMedian          float64
latitudeLeft         int32
latitudeRight        int32
longitudeLeft        int32
longitudeRight       int32
point1              object
point2              object
point3              object
point4              object
point5              object
dtype: object

In [136]:
dfLatLong['year'] = dfLatLong['year'].apply(np.int64)
dfLatLong

,year,latitudeSquare,longitudeSquare,sstMin,sstMax,sstMean,sstMedian,latitudeLeft,latitudeRight,longitudeLeft,longitudeRight,point1,point2,point3,point4,point5
1,1851,-100,-90,19.4,26.7,19.4,26.7,0,10,-100,-90,"0, -100","0, -90","10, -90","10, -100","0, -100"
2,1851,-90,-80,21.7,27.8,21.7,27.8,0,10,-90,-80,"0, -90","0, -80","10, -80","10, -90","0, -90"
3,1851,-80,-70,25.0,28.9,25.0,28.9,0,10,-80,-70,"0, -80","0, -70","10, -70","10, -80","0, -80"
5,1851,-60,-50,26.1,28.3,26.1,28.3,0,10,-60,-50,"0, -60","0, -50","10, -50","10, -60","0, -60"
6,1851,-100,-90,24.4,28.3,24.4,28.3,10,20,-100,-90,"10, -100","10, -90","20, -90","20, -100","10, -100"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,1857,-70,-60,0.6,29.4,0.6,29.4,30,40,-70,-60,"30, -70","30, -60","40, -60","40, -70","30, -70"
170,1857,-60,-50,10.0,28.3,10.0,28.3,30,40,-60,-50,"30, -60","30, -50","40, -50","40, -60","30, -60"
173,1857,-80,-70,1.1,26.1,1.1,26.1,40,50,-80,-70,"40, -80","40, -70","50, -70","50, -80","40, -80"
174,1857,-70,-60,-2.8,26.7,-2.8,26.7,40,50,-70,-60,"40, -70","40, -60","50, -60","50, -70","40, -70"


In [121]:
dfFinal.dtypes

year                 int64
latitudeSquare      object
longitudeSquare     object
sstMin             float64
sstMax             float64
sstMean            float64
sstMedian          float64
latitudeLeft         int32
latitudeRight        int32
longitudeLeft        int32
longitudeRight       int32
dtype: object